In [ ]:
import tkinter as tk
from tkinter import messagebox
import random

# Helper functions
def extract_rank(card):
    return card[:-1]

def extract_suit(card):
    return card[-1]

def is_flush(cards):
    suits = [extract_suit(card) for card in cards]
    return len(set(suits)) == 1

def is_straight(cards):
    rank_order = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    ranks = [rank_order[extract_rank(card)] for card in cards]
    ranks = sorted(ranks)
    if len(set(ranks)) == 5 and (ranks[4] - ranks[0] == 4):
        return True
    if set(ranks) == {14, 2, 3, 4, 5}:
        return True
    return False

def kind(n, ranks):
    for rank in set(ranks):
        if ranks.count(rank) == n:
            return rank
    return None

def evaluate_hand(cards):
    rank_order = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    ranks = [rank_order[extract_rank(card)] for card in cards]
    if is_straight(cards) and is_flush(cards):
        return (9, max(ranks))
    if kind(4, ranks):
        return (8, kind(4, ranks))
    three_of_a_kind = kind(3, ranks)
    pair = kind(2, ranks)
    if three_of_a_kind and pair:
        return (7, three_of_a_kind, pair)
    if is_flush(cards):
        return (6, sorted(ranks, reverse=True))
    if is_straight(cards):
        return (5, max(ranks))
    if three_of_a_kind:
        return (4, three_of_a_kind)
    if kind(2, ranks):
        pairs = [rank for rank in ranks if ranks.count(rank) == 2]
        pairs = list(set(pairs))
        if len(pairs) == 2:
            return (3, max(pairs), min(pairs))
        else:
            ranks.remove(pairs[0])
            return (2, pairs[0], max(ranks))
    return (1, sorted(ranks, reverse=True))

def monte_carlo_simulation(hole_cards, community_cards, iterations=10000):
    ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    suits = ['h', 'c', 'd', 's']
    all_cards = [f"{rank}{suit}" for rank in ranks for suit in suits]
    
    remaining_cards = [card for card in all_cards if card not in hole_cards and card not in community_cards]
    win_count = 0
    for _ in range(iterations):
        board = community_cards + random.sample(remaining_cards, 5 - len(community_cards))
        remaining_for_opponent = [card for card in remaining_cards if card not in board]
        opponent_hole_cards = random.sample(remaining_for_opponent, 2)
        our_hand_value = evaluate_hand(hole_cards + board)
        opponent_hand_value = evaluate_hand(opponent_hole_cards + board)
        if our_hand_value > opponent_hand_value:
            win_count += 1
    win_percentage = (win_count / iterations) * 100
    return win_percentage


class PokerOddsCalculatorUpdated:
    def __init__(self, root):
        self.root = root
        self.root.title("Poker Odds Calculator")
        self.suits = {
            'h': 'red',
            'd': 'red',
            'c': 'black',
            's': 'black'
        }
        self.ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.selected_cards = []
        self.community_cards = []
        for i, rank in enumerate(self.ranks):
            for suit, color in self.suits.items():
                card_text = rank + suit
                btn = tk.Button(self.root, text=card_text, fg=color, command=lambda card=card_text: self.select_card(card))
                btn.grid(row=i, column=list(self.suits.keys()).index(suit))
        self.selected_display = tk.Label(self.root, text="", font=("Arial", 16))
        self.selected_display.grid(row=14, column=0, columnspan=4)
        self.community_display = tk.Label(self.root, text="", font=("Arial", 16))
        self.community_display.grid(row=15, column=0, columnspan=4)
        self.select_mode = tk.StringVar(value="personal")
        self.toggle_btn = tk.Button(self.root, text="Toggle Selection (Personal/Community)", command=self.toggle_selection_mode)
        self.toggle_btn.grid(row=16, column=0, columnspan=4)
        self.calculate_btn = tk.Button(self.root, text="Calculate Odds", command=self.calculate_odds)
        self.calculate_btn.grid(row=17, column=0, columnspan=4)
        self.odds_display = tk.Label(self.root, text="", font=("Arial", 18))
        self.odds_display.grid(row=18, column=0, columnspan=4)
        self.refresh_btn = tk.Button(self.root, text="Refresh", command=self.reset_selection)
        self.refresh_btn.grid(row=19, column=0, columnspan=4)

    
        
    def select_card(self, card):
        if self.select_mode.get() == "personal":
            if card not in self.selected_cards and len(self.selected_cards) < 2:
                self.selected_cards.append(card)
                self.selected_display["text"] = "Your Cards: " + " ".join(self.selected_cards)
            elif card in self.selected_cards:
                self.selected_cards.remove(card)
                self.selected_display["text"] = "Your Cards: " + " ".join(self.selected_cards)
        elif self.select_mode.get() == "community":
            if card not in self.community_cards and len(self.community_cards) < 5:
                self.community_cards.append(card)
                self.community_display["text"] = "Community Cards: " + " ".join(self.community_cards)
            elif card in self.community_cards:
                self.community_cards.remove(card)
                self.community_display["text"] = "Community Cards: " + " ".join(self.community_cards)
                
    def toggle_selection_mode(self):
        if self.select_mode.get() == "personal":
            self.select_mode.set("community")
            self.toggle_btn["text"] = "Toggle Selection (Community/Personal)"
        else:
            self.select_mode.set("personal")
            self.toggle_btn["text"] = "Toggle Selection (Personal/Community)"
            
    def calculate_odds(self):
        print("Calculate Odds button pressed.")
        if len(self.selected_cards) != 2:
            messagebox.showerror("Error", "Please select exactly 2 personal cards.")
            return

        try:
            odds = monte_carlo_simulation(self.selected_cards, self.community_cards)
            print(f"Calculated Odds: {odds:.2f}%")
            self.odds_display["text"] = f"Winning Odds: {odds:.2f}%"
        except Exception as e:
            print(f"Error during Monte Carlo simulation: {e}")
            messagebox.showerror("Error", f"An error occurred during the odds calculation. Please check the console for details.")
    
    def reset_selection(self):
        self.selected_cards.clear()
        self.community_cards.clear()
        self.selected_display["text"] = ""
        self.community_display["text"] = ""
        self.odds_display["text"] = ""
        self.select_mode.set("personal")  # Reset to personal card selection mode
        self.toggle_btn["text"] = "Toggle Selection (Personal/Community)"


if __name__ == "__main__":
    root = tk.Tk()
    app = PokerOddsCalculatorUpdated(root)
    root.mainloop()
